In [1]:
import syft as sy
from torch import nn, optim
import torch as th

W0714 18:17:01.096807 4321358720 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'


W0714 18:17:01.124337 4321358720 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
# Dataset initialisation
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [3]:
# VirtualWorkers initialisation
hook = sy.TorchHook(th)

bob     = sy.VirtualWorker(hook, id='bob')
alice   = sy.VirtualWorker(hook, id='alice')
central = sy.VirtualWorker(hook, id='central')

In [4]:
# Data sending
bob.clear_objects()
alice.clear_objects()
central.clear_objects()

data_bob   = data[:2].send(bob)
data_alice = data[2:].send(alice)

target_bob   = target[:2].send(bob)
target_alice = target[2:].send(alice)

print('bob before:',     bob._objects)
print('alice before:',   alice._objects)
print('central before:', central._objects)

bob before: {81965632757: tensor([[1., 1.],
        [0., 1.]], requires_grad=True), 30760523774: tensor([[1.],
        [1.]], requires_grad=True)}
alice before: {72883206436: tensor([[1., 0.],
        [0., 0.]], requires_grad=True), 4994224652: tensor([[0.],
        [0.]], requires_grad=True)}
central before: {}


In [5]:
def train(_data, _target, iterations=20):
    # send model to the data
    model = nn.Linear(2, 1).copy().send(_data.location)
    
    opt = optim.SGD(params=model.parameters(), lr=0.1)

    for iter in range(iterations):            
        # do normal training
        opt.zero_grad()
        pred = model(_data)
                    
        loss = ((pred - _target)**2).sum()
        loss.backward()
                    
        opt.step()
        print(loss.get().data)

    model.move(central)

train(data_bob, target_bob)
print('bob:', bob._objects)
print('central:', central._objects)
train(data_alice, target_alice)
print('alice:', alice._objects)
print('central:', central._objects)

tensor(10.2982)
tensor(0.0798)
tensor(0.0011)
tensor(0.0004)
tensor(0.0004)
tensor(0.0003)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(8.3802e-05)
tensor(6.9749e-05)
tensor(5.8053e-05)
tensor(4.8318e-05)
tensor(4.0215e-05)
tensor(3.3471e-05)
tensor(2.7859e-05)
tensor(2.3188e-05)
bob: {81965632757: tensor([[1., 1.],
        [0., 1.]], requires_grad=True), 30760523774: tensor([[1.],
        [1.]], requires_grad=True)}
central: {47636388032: Parameter containing:
tensor([[-0.0062,  0.5717]], requires_grad=True), 12871416100: Parameter containing:
tensor([0.4318], requires_grad=True)}
tensor(0.1510)
tensor(0.0384)
tensor(0.0122)
tensor(0.0057)
tensor(0.0038)
tensor(0.0030)
tensor(0.0025)
tensor(0.0021)


tensor(0.0018)
tensor(0.0016)
tensor(0.0013)
tensor(0.0011)
tensor(0.0010)
tensor(0.0008)
tensor(0.0007)
tensor(0.0006)
tensor(0.0005)
tensor(0.0004)
tensor(0.0004)
tensor(0.0003)
alice: {72883206436: tensor([[1., 0.],
        [0., 0.]], requires_grad=True), 4994224652: tensor([[0.],
        [0.]], requires_grad=True)}
central: {47636388032: Parameter containing:
tensor([[-0.0062,  0.5717]], requires_grad=True), 12871416100: Parameter containing:
tensor([0.4318], requires_grad=True), 73691493462: Parameter containing:
tensor([[ 0.0227, -0.6195]], requires_grad=True), 779146013: Parameter containing:
tensor([-0.0140], requires_grad=True)}


In [11]:
# calculate the result on central server
def model():
    model = nn.Linear(2, 1)
    
    print('weight before: ', model.weight)
    print('bias before: ', model.bias)
    
    length = len(central._objects) // 2
    
    weight = th.Tensor(1, 2)
    bias   = th.Tensor(1)
    
    for  _, value in central._objects.items():
        if value.size() == weight.size():
            weight += value
        else:
            bias += value
    
    weight /= length
    bias   /= length
    
    with th.no_grad():
        model.weight.set_(weight)
        model.bias.set_(bias)
    
    print('weight after: ', model.weight)
    print('bias after: ', model.bias)
    
    return model

model()

weight before:  Parameter containing:
tensor([[-0.4742,  0.3885]], requires_grad=True)
bias before:  Parameter containing:
tensor([-0.6204], requires_grad=True)
weight after:  Parameter containing:
tensor([[ 0.0083, -0.0239]], requires_grad=True)
bias after:  Parameter containing:
tensor([0.2089], requires_grad=True)


Linear(in_features=2, out_features=1, bias=True)